import required libraries


In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer as ps
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df=pd.read_csv('/content/spam.csv',encoding='latin-1')

rename column names as label and message

In [ ]:
df=df.rename(columns={'v1':'label','v2':'message'})

In [ ]:
df.head()

,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


remove unwanted columns

In [ ]:
df=df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [ ]:
df.shape

(5572, 2)

In [ ]:
df.isnull().sum()

label      0
message    0
dtype: int64

check for any duplicated data and remove the duplicates

In [ ]:
df.duplicated().sum()

403

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

(5169, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5169 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5169 non-null   object
 1   message  5169 non-null   object
dtypes: object(2)
memory usage: 121.1+ KB


remove all the unwanted characters(symbols,special characters and numbers)in the message except a-z and A-Z

In [ ]:
corpus=[]
for i in range(0, len(df)):
  changed=re.sub('[^a-zA-Z]',' ',df.message.iloc[i])
  changed=changed.lower()
  changed=changed.split()
  changed=[ps().stem(word) for word in changed if not word in stopwords.words('english')]
  changed=' '.join(changed)
  corpus.append(changed)

feature extraction through count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
X=cv.fit_transform(corpus).toarray()

In [ ]:
y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

train the model using Multinomial Naive Bayes algorithm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion=confusion_matrix(y_test,y_pred)
print(confusion)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

prediction

In [ ]:

def predict_message(message):
    message_counts = cv.transform([message])
    prediction = model.predict(message_counts)
    if prediction[0]==0:
        return 'ham'
    else:
        return 'spam'

In [ ]:
new_message="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
print(predict_message(new_message))